In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, udf, col
from pyspark.sql.types import *
import requests
import json
from pyspark import SparkConf, SparkContext
conf = SparkConf().set("spark.jars", "/home/glue_user/workspace/system/commons-pool2-2.11.1.jar")
sc = SparkContext(conf=conf)
spark = SparkSession(sc) \
    .builder \
    .appName("KafkaReaderWithAvro") \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,"
            "org.apache.spark:spark-avro_2.12:3.3.0") \
    .getOrCreate()

# Kafka parameters
bootstrap_servers = 'broker:29092'  # Kafka server
topic = 'topic_prefix.kafkaDB.users' 
kafka_bootstrap_servers = bootstrap_servers
kafka_topic = topic


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Cannot run multiple SparkContexts at once; existing SparkContext(app=livy-session-5, master=local) created by __init__ at /tmp/5606982136895303830:600 
Traceback (most recent call last):
  File "/home/glue_user/spark/python/pyspark/context.py", line 195, in __init__
    SparkContext._ensure_initialized(self, gateway=gateway, conf=conf)
  File "/home/glue_user/spark/python/pyspark/context.py", line 438, in _ensure_initialized
    raise ValueError(
ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=livy-session-5, master=local) created by __init__ at /tmp/5606982136895303830:600 



In [2]:
spark.sparkContext._jsc.sc().listJars()




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

JavaObject id=o67

In [7]:
schema_registry_url = "https://rnung-103-224-144-138.a.free.pinggy.link"

# Broadcast the schema fetcher
def get_schema(schema_id):
    url = f"{schema_registry_url}/schemas/ids/{schema_id}"
    response = requests.get(url)
    response.raise_for_status()
    return json.loads(response.json()["schema"])

# Fetch the schema only once (assuming it's static)
example_schema_id = 3  # replace with your actual schema ID or dynamically parse it
parsed_schema = get_schema(example_schema_id)

# Broadcast schema for UDF use
broadcast_schema = spark.sparkContext.broadcast(parsed_schema)

# UDF to decode Avro messages
def decode_avro(avro_bytes):
    try:
        if avro_bytes[0] != 0:
            return None
        schema_id = int.from_bytes(avro_bytes[1:5], byteorder='big')
        payload = BytesIO(avro_bytes[5:])
        return json.dumps(schemaless_reader(payload, broadcast_schema.value))
    except Exception as e:
        return json.dumps({"error": str(e)})

decode_avro_udf = udf(decode_avro, StringType())
# print(parsed_schema)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Read data from Kafka
kafka_df = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .load()

# Decode value column
decoded_df = kafka_df.withColumn("decoded_value", decode_avro_udf(col("value")))

# Show results
decoded_df.select("decoded_value").show(truncate=False)

In [13]:
from pyspark.sql import SparkSession

# Assuming SparkSession is already created as `spark`
kafka_df = spark.read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "earliest") \
    .load()

# Collect the records
kafka_records = kafka_df.collect()

# Iterate through each record
for record in kafka_records:
    key = record["key"]
    value = record["value"]
    topic = record["topic"]
    partition = record["partition"]
    offset = record["offset"]

    print(f"Key: {key}, Value: {value}, Topic: {topic}, Partition: {partition}, Offset: {offset}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o137.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (6aa2fbc3dead executor driver): java.lang.NoClassDefFoundError: Could not initialize class org.apache.spark.sql.kafka010.consumer.KafkaDataConsumer$
	at org.apache.spark.sql.kafka010.KafkaSourceRDD.compute(KafkaSourceRDD.scala:72)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apa